# Estimación del estadio puberal de Tanner para añadir al M5 y afinarlo 

In [3]:
import pandas as pd
import pymc as pm
import arviz as az
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import pytensor  
from patsy import dmatrix


In [4]:
# read data
df = pd.read_csv("../data/clean/clean2_final_nutricion_salud.csv")

---

In [5]:
# Filtramos observaciones con datos completos de talla, edad y sexo
df_model = df[["talla_cm", "edad_anios_calc", "sexo", "peso_kg", "municipio", "id_persona"]].dropna()

# Establecemos 'id_persona' como índice del DataFrame
df_model = df_model.set_index("id_persona")

# Codificamos correctamente sexo 
# Recodeamos sexo: 0 = mujer, 1 = hombre
df_model["sexo"] = df_model["sexo"].map({2: 0, 1: 1})

# Confirmamos los tipos de variables y que no haya NAs
df_model.info()
df_model.describe()
df_model["sexo"].value_counts()
df_model["municipio"].nunique()

<class 'pandas.core.frame.DataFrame'>
Index: 25355 entries, 100001_3 to 70336_4
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   talla_cm         25355 non-null  float64
 1   edad_anios_calc  25355 non-null  float64
 2   sexo             25355 non-null  int64  
 3   peso_kg          25355 non-null  float64
 4   municipio        25355 non-null  float64
dtypes: float64(4), int64(1)
memory usage: 1.2+ MB


177

In [6]:
# Codificamos cada municipio con un índice entero único (de 0 a n_grupos - 1)
df_model["municipio_idx"] = pd.Categorical(df_model["municipio"]).codes

# Verificamos que se creó correctamente
df_model[["municipio", "municipio_idx"]].drop_duplicates().sort_values("municipio_idx").head()


,municipio,municipio_idx
id_persona,,
100001_3,1.0,0
11276_3,2.0,1
110044_5,3.0,2
100075_5,4.0,3
100121_6,5.0,4


---

In [7]:
# Centrar la edad y guardar la media: 
# Esto ayuda a que el intercepto sea interpretable (talla promedio en la edad media) y mejora la estabilidad numérica del muestreo.
edad_mean = df_model["edad_anios_calc"].mean()
df_model["edad_c"] = df_model["edad_anios_calc"] - edad_mean

print(f"Edad media en la muestra: {edad_mean:.3f} años")
df_model[["edad_anios_calc", "edad_c"]].head()


Edad media en la muestra: 6.266 años


,edad_anios_calc,edad_c
id_persona,,
100001_3,8.695414,2.429254
100006_6,11.211499,4.945339
100008_10,7.561944,1.295784
100009_3,6.277892,0.011732
100010_7,8.145106,1.878946


---

## Modelo M5: splines 

In [8]:
# Construcción de B-splines cúbicos para la edad
# (A) Hiperparámetro de flexibilidad
df_spline = 5        # Empezamos con 5; luego probamos 6 o 7 si hace falta.

# (B) Matriz de B-splines cúbicos de la edad (sin intercepto)
X_spline = dmatrix(
    "bs(edad, df=df_spline, degree=3, include_intercept=False) - 1",
    {"edad": df_model["edad_anios_calc"].values},
    return_type="dataframe"
)

# (C) A NumPy para PyMC
X_s = X_spline.to_numpy()

# (D) Chequeo rápido
n_obs, n_s = X_s.shape
print("X_s shape:", X_s.shape)   # (n_obs, n_bases)


X_s shape: (25355, 5)


In [9]:
# Variable objetivo y predictores “clásicos”
y        = df_model["talla_cm"].to_numpy()
sexo     = df_model["sexo"].to_numpy().astype(int)  # 0 = mujer, 1 = hombre

# Edad centrada (importante para estabilidad y para interpretar el intercepto)
edad_c   = (df_model["edad_anios_calc"] - df_model["edad_anios_calc"].mean()).to_numpy()

# Índice entero de municipio por observación (0..n_muni-1)
muni_idx = df_model["municipio_idx"].to_numpy().astype(int)

# Tamaños
n_obs_chk = y.shape[0]
n_muni    = int(muni_idx.max()) + 1

# Chequeos
assert X_s.shape[0] == n_obs_chk, "X_s debe tener tantas filas como observaciones."
print(f"n_obs={n_obs_chk} | n_s(bases)={n_s} | n_muni={n_muni}")


n_obs=25355 | n_s(bases)=5 | n_muni=177


In [10]:

with pm.Model() as modelo_m5:
    # --- Priors globales (cm) ---
    beta_0        = pm.Normal("beta_0", 125, 10)     # talla a la edad media (sexo=0)
    beta_sexo     = pm.Normal("beta_sexo", 3, 2)     # diferencia nivel hombre vs mujer
    beta_edad_sex = pm.Normal("beta_edad_sex", 0, 2) # interacción lineal edad_c × sexo (opcional)

    # --- Pesos de las bases spline (regularización suave) ---
    # Normal(0,1) ≈ “ridge” suave; si vemos curvas muy onduladas, bajamos a 0.5
    w_s = pm.Normal("w_s", 0, 1.0, shape=n_s)

    # --- Efectos aleatorios por municipio (no-centrados) ---
    sd_a0 = pm.HalfNormal("sd_a0", 5)   # sd de interceptos municipales (cm)
    sd_a1 = pm.HalfNormal("sd_a1", 2)   # sd de pendientes municipales (cm/año)

    z_a0 = pm.Normal("z_a0", 0, 1, shape=n_muni) # de la centralización
    z_a1 = pm.Normal("z_a1", 0, 1, shape=n_muni) # de la centralización
    a0   = pm.Deterministic("a0", sd_a0 * z_a0) 
    a1   = pm.Deterministic("a1", sd_a1 * z_a1)

    # --- Curva de edad a partir de las bases ---
    # dot(X_s, w_s): vector (n_obs,) con la contribución no lineal de la edad
    f_edad = pm.Deterministic("f_edad", pm.math.dot(X_s, w_s))

    # --- Media (predicción) ---
    mu = (
        beta_0
        + beta_sexo * sexo
        + f_edad                        # curva suave común por edad
        + a0[muni_idx]                  # nivel municipal
        + a1[muni_idx] * edad_c         # “tilt” lineal municipal
        + beta_edad_sex * (edad_c * sexo)  # opcional: diferencia de pendiente global por sexo
    )

    # --- Likelihood robusta ---
    nu    = pm.Exponential("nu", 1/10)     # g.l. (media 10)
    sigma = pm.HalfNormal("sigma_obs", 5)  # escala residual (cm)
    talla_obs = pm.StudentT("talla_obs", nu=nu, mu=mu, sigma=sigma, observed=y)


Por qué “no centrado” en a0, a1:

Cuando la variación entre municipios (sd_a*) es pequeña o el muestreo está “tenso”, la forma no centrada (z ~ N(0,1); a = sd*z) mejora mucho la mezcla y reduce divergencias.

In [ ]:
with modelo_m5:
    idata_m5 = pm.sample(
        draws=1000, tune=1000,          # empezamos moderado; si todo OK luego subimos a 1200–1500
        chains=3, cores=1,            # cores=1 evita issues en notebooks/entornos limitados
        target_accept=0.97,           # alto = menos divergences (si hay, subimos a 0.99)
        init="jitter+adapt_diag",
        random_seed=42,
        return_inferencedata=True,
        idata_kwargs={"log_likelihood": True}
    )

    # PPC (para plot_ppc y checks)
    ppc_m5 = pm.sample_posterior_predictive(idata_m5, return_inferencedata=True, random_seed=42)
    idata_m5.extend(ppc_m5)   # <- sin reasignar

    print(idata_m5.groups())


Initializing NUTS using jitter+adapt_diag...
Sequential sampling (3 chains in 1 job)
NUTS: [beta_0, beta_sexo, beta_edad_sex, w_s, sd_a0, sd_a1, z_a0, z_a1, nu, sigma_obs]


/opt/anaconda3/envs/pymc_env/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Sampling 3 chains for 800 tune and 800 draw iterations (2_400 + 2_400 draws total) took 908 seconds.
We recommend running at least 4 chains for robust computation of convergence diagnostics
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Sampling: [talla_obs]


/opt/anaconda3/envs/pymc_env/lib/python3.10/site-packages/rich/live.py:256: UserWarning: install "ipywidgets" for 
Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

['posterior', 'posterior_predictive', 'log_likelihood', 'sample_stats', 'observed_data']


In [12]:
summ = az.summary(idata_m5, round_to=2)
display(summ.head(12))

/opt/anaconda3/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:596: RuntimeWarning: invalid value encountered in scalar divide
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)
/opt/anaconda3/envs/pymc_env/lib/python3.10/site-packages/arviz/stats/diagnostics.py:991: RuntimeWarning: invalid value encountered in scalar divide
  varsd = varvar / evar / 4


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta_0,106.30,0.37,105.65,106.97,0.01,0.01,617.49,1076.07,1.00
beta_sexo,0.48,0.07,0.36,0.61,0.00,0.00,4289.96,1814.74,1.00
beta_edad_sex,-0.24,0.02,-0.27,-0.19,0.00,0.00,4278.82,1915.08,1.00
w_s[0],10.42,0.36,9.77,11.11,0.01,0.01,3987.90,1836.94,1.00
w_s[1],10.55,0.30,10.01,11.11,0.01,0.01,1364.69,1717.18,1.00
w_s[2],3.91,0.48,2.92,4.77,0.01,0.01,1277.27,1477.61,1.00
w_s[3],-1.19,0.61,-2.29,-0.00,0.02,0.01,1051.69,1530.38,1.00
w_s[4],-4.59,0.66,-5.82,-3.35,0.02,0.01,976.31,1477.05,1.00
z_a0[0],0.76,0.14,0.51,1.03,0.01,0.00,486.27,972.86,1.01
z_a0[1],0.53,0.14,0.26,0.79,0.01,0.00,558.92,987.35,1.01


---

## Estimación estadio puberal Tanner con modelos latentes probabilísticos

Qué son los estadios Tanner?

Se usan clínicamente para evaluar el desarrollo físico en la pubertad, con 5 fases (Tanner 1 a 5), basadas en características sexuales secundarias: desarrollo mamario/genital y vello púbico. Son el “estándar de oro” en pediatría

### Modelos latentes probabilísticos aplicados a Tanner

Un modelo latente parte de la idea de que, además de las variables que observamos (edad, sexo, talla, IMC), existe una **variable oculta** que no se mide directamente pero que explica parte del patrón en los datos.  
En el contexto de crecimiento infantil, esa variable oculta corresponde al **estadio puberal de Tanner (I–V)**.

#### 1. Probabilidades de estadio
El modelo no asigna a cada niño un estadio fijo, sino que estima **probabilidades** para cada uno de los cinco estadios.  
Estas probabilidades dependen de características observadas como la edad, el sexo y el IMC.  
Ejemplo: un niño de 13 años podría tener:  
- 10% Tanner II  
- 40% Tanner III  
- 50% Tanner IV  

#### 2. Distribuciones por estadio
Cada estadio Tanner tiene su propia **distribución de talla esperada**.  
- Tanner I: estaturas más bajas.  
- Tanner V: estaturas más altas.  
- Los estadios intermedios representan el crecimiento acelerado del estirón puberal.  

De esta forma, si un niño está en Tanner III, se espera que su talla se distribuya alrededor de un valor típico para ese estadio, con cierta variabilidad natural.

#### 3. Mezcla probabilística
Como no sabemos en qué estadio se encuentra exactamente cada niño, el modelo combina todas las distribuciones ponderadas por las probabilidades:  

**Prob(talla) = Prob(Tanner I)·Distribución I + Prob(Tanner II)·Distribución II + ... + Prob(Tanner V)·Distribución V**

Esto refleja que la talla observada no proviene de una única categoría fija, sino de una mezcla que captura la **incertidumbre** del estado puberal.

#### 4. Interpretación
- Las **probabilidades** responden a: “¿Qué tan probable es que este niño esté en cada estadio Tanner?”.  
- Las **distribuciones por estadio** responden a: “Si el niño estuviera en este estadio, ¿cuál sería la talla típica?”.  
- La **mezcla final** responde a: “Dada su edad y características, ¿cómo se distribuye la talla observada?”.  

#### Ventajas
- No fuerza a clasificar en un único estadio.  
- Captura la incertidumbre en edades de transición.  
- Representa mejor la heterogeneidad real del crecimiento que un modelo lineal simple.  
- Permite aproximar el Tanner aunque no haya mediciones clínicas directas.


---

## Sandbox

In [14]:
df.columns.tolist()

['folio',
 'intp',
 'entidad',
 'municipio',
 'localidad',
 'sexo',
 'edad_meses',
 'edad_anios',
 'peso_kg',
 'talla_cm',
 'zscore_imc_para_edad',
 'clasificacion_imc',
 'ponderador',
 'region',
 'id_persona',
 'num_integrante',
 'edad_dias',
 'fecha_nacimiento',
 'fecha_visita',
 'altitud_localidad',
 'code_upm',
 'estrato_diseño',
 'estrato_urbanidad',
 'estrato_marginalidad',
 'estrato_varianza',
 'area_urbana_rural',
 'region_geo',
 'ageb',
 'indice_socioeconomico',
 'afiliacion_salud',
 'nivel_socioecon_decil',
 'nivel_socioecon_quintil',
 'nivel_socioecon_tercil',
 'ponderador_muestra',
 'tipo_hemocue',
 'hemoglobina_g_dl',
 'hemoglobina_ajustada_g_dl',
 'anemia',
 'folio_consecutivo',
 'munici',
 'locali',
 'edad',
 'dia_nac',
 'mes_nac',
 'anio_nac',
 'est_urb',
 'est_marg',
 'pondef',
 'folio_vivienda',
 'meses',
 'afilia_1ra',
 'afilia_tras',
 'pondei',
 'deciles',
 'pondeh',
 'grupo_edad',
 'imc',
 'edad_anios_calc']

### Explicaciones del modelo jerárquico bayesiano con splines y mejoras

### 1) Uso de splines para la edad
Una recta simple asume que la relación entre edad y talla es constante en todo el rango de edades. En el crecimiento infantil, especialmente entre los 8 y 18 años, esto no es cierto debido al “growth spurt” o estirón puberal, donde la velocidad de crecimiento cambia rápidamente.  
Los splines, en particular los B-splines cúbicos, dividen la edad en tramos conectados suavemente. Esto permite que la curva se adapte a aceleraciones y desaceleraciones en el crecimiento sin imponer una forma rígida. Además, controlando el número de tramos, se evita sobreajuste y se mantiene la suavidad.

---

### 2) Pendiente aleatoria de edad por municipio
Si solo incluimos interceptos aleatorios, cada municipio tendrá un nivel medio distinto de talla, pero se asumirá la misma relación con la edad.  
Permitir pendientes aleatorias significa que la velocidad de crecimiento puede variar entre municipios. Esto refleja diferencias geográficas y contextuales, haciendo el modelo más realista y flexible.

---

### 3) Student-t en vez de Normal
La distribución Normal, usada como verosimilitud, es sensible a valores atípicos. En datos reales de talla pueden existir mediciones erróneas o casos extremos.  
La distribución Student-t tiene colas más pesadas, lo que reduce el impacto de los outliers. Si los datos no presentan valores extremos, el modelo ajustará automáticamente los grados de libertad y se parecerá a la Normal.

---

### 4) Centrados de variables
Centrar la edad significa restarle su media para que el cero de la variable corresponda a la edad promedio de la muestra. Esto facilita la interpretación del intercepto como la talla media a la edad media y mejora la eficiencia del muestreo MCMC.  
Podemos centrar otras variables si fuera necesario, pero en este caso mantener el sexo como binario 0/1 hace que la interpretación de su coeficiente sea más directa.

---

### 5) Interacción entre edad y sexo
Aunque los splines capturan la forma general del crecimiento, niños y niñas no siguen el mismo patrón: el estirón suele ocurrir antes en las niñas y durar más en los niños.  
Agregar un término de interacción entre edad y sexo permite que el modelo capture diferencias globales en la pendiente de crecimiento entre ambos sexos, manteniendo la parte curva compartida.

---

### 6) Evaluación con prior predictive y posterior predictive
El prior predictive consiste en simular datos solo con las distribuciones a priori, sin usar datos reales, para verificar que las suposiciones iniciales producen valores plausibles.  
El posterior predictive simula datos a partir de los parámetros ajustados, para evaluar si el modelo reproduce bien los patrones observados. Estas comprobaciones son fundamentales para validar que el modelo es coherente y útil.

---

### 7) LOO/WAIC para comparar variantes
Cuando se tengan diferentes versiones del modelo (por ejemplo, con y sin interacción, con más o menos nodos spline), se puede usar LOO (Leave-One-Out cross-validation) o WAIC (Widely Applicable Information Criterion) para comparar su ajuste y capacidad predictiva.  
Estos criterios penalizan la complejidad innecesaria y ayudan a elegir el modelo más parsimonioso que mantenga buen rendimiento.

---

### 8) Comparación con curvas CDC
Se pueden calcular percentiles como el P50 (mediana) y otros por sexo y edad, y compararlos con las curvas de referencia del CDC para identificar sesgos sistemáticos (por ejemplo, diferencias en el nivel o en la forma de la curva).

---

### 9) Proxy para estadio puberal de Tanner ML, Modelos Latentes Probabilísticos u otro (TBD)


---